In [ ]:
# | default_exp social_image_generator

In [ ]:
# | export

from typing import *
from pathlib import Path
import re
import asyncio
import shutil
from tempfile import TemporaryDirectory

import openai
import typer
from playwright.async_api import async_playwright
from ruamel.yaml import YAML

from nbdev_mkdocs._helpers.utils import set_cwd, get_value_from_config
from nbdev_mkdocs._package_data import get_root_data_path

In [ ]:
# | export

def _generate_ai_image(prompt: str, n: int = 1, size: str = "512x512") -> str:
    """Generate an image for social card using the OpenAI Image API.
    
    Args:
        prompt: The prompt to use for generating the image.
        n: The number of images to generate (default 1).
        size: The size of the image to generate (default "1024x1024").
        
    Returns:
        The URL of the generated image.
    """
    try:
        response = openai.Image.create(
          prompt=prompt,
          n=n,
          size=size
        )
        image_url = response['data'][0]['url']
    
    except Exception as e:
        typer.echo(f"Request to OpenAI failed: {e}")
        typer.echo("Using the default social image.")
        
        image_url = "default_social_logo.png"
        
    return image_url

In [ ]:
prompt = "cute animal, browsing computer, high tech, flat, digital art"

image_url = _generate_ai_image(prompt=prompt)
image_url

Request to OpenAI failed: Billing hard limit has been reached
Using the default social image.


'default_social_logo.png'

In [ ]:
# | export

def _generate_html_str(root_path: str, image_url: str) -> str:
    """Generate html string for the social card
    
    Args:
        root_path: The root path of the project.
        image_url: The image URL to be included in the HTML.
    """
    
    with set_cwd(root_path):
        
        _custom_social_image_template_path = get_root_data_path() / "custom-social-image-template.html"
        
        with open(_custom_social_image_template_path, "r") as f:
            _html_template = f.read()
        
        author_name = get_value_from_config(root_path, "author")
        project_name = get_value_from_config(root_path, "repo")
        project_description = get_value_from_config(root_path, "description")
    
        d = dict(
            author_name=author_name, 
            project_name=project_name, 
            project_description=project_description, 
            image_url=image_url
        )
        
        return _html_template.format(**d)

In [ ]:

with TemporaryDirectory() as d:
    
    shutil.copyfile(Path("..") / "settings.ini", Path(d) / "settings.ini")
    
    author_name = get_value_from_config(d, "author")
    project_name = get_value_from_config(d, "repo")
    project_description = get_value_from_config(d, "description")

    
    image_url = "https://sample-image.png"
    actual = _generate_html_str(d, image_url)
    print(actual)
    
    assert f"{author_name}/" in actual
    assert project_name in actual
    assert image_url in actual

<!DOCTYPE html>
<html>
   <head>
     <link href='https://fonts.googleapis.com/css?family=Source Sans Pro' rel='stylesheet'/>
   </head>
   <body style="margin: 0px;height: 640px;width: 1280px;overflow: hidden;font-family: 'Source Sans Pro';">
      <div id="container" style="
         overflow: hidden;
         height: 640px;
         width: 1280px;
         ">
         <div style="width: 670px;float:left;height: 640px;">
            <h1 style="padding-top: 110px;padding-left: 80px;font-size: 85px;color: #2E363D;margin: 0;"><span style="
               font-weight: 100;
               ">airt/</span>nbdev-mkdocs</h1>
            <p style="padding-top: 70px;padding-left: 80px;font-size: 30px;color: #6E7681;">Extension of nbdev for generating documentation using Material for Mkdocs instead of Quarto</p>
         </div>
         <div style="width: 555px;float:left;height: 640px;overflow: hidden;padding: 53px 53px 53px 0px;">
            <img src="https://sample-image.png" style="
        

In [ ]:
# | export

async def _capture_and_save_screenshot(src_path: str, dst_path: str):
    """Capture screenshot of an HTML file from source directory and save the
    output in destination directory
    
    Args:
        src_path: The source path of the HTML file that will be used to generate the PNG image.
        dst_path: The destination path where the generated screenshot image will be saved.
    """
    playwright = await async_playwright().start()
    browser = await playwright.chromium.launch()
    page = await browser.new_page()

    html_path = Path(src_path) / "social_image.html"
    await page.goto(f'file://{str(html_path.resolve())}')

    png_path = Path(dst_path) / "mkdocs" / "docs_overrides" / "images" / "social_image.png"
    await page.screenshot(path = str(png_path.resolve()))
    await browser.close()

In [ ]:
with TemporaryDirectory() as d:
    
    html_path = Path(d) / "social_image.html"
    with open(html_path, "w") as f:
        f.write("<html><body><p>This is a sample text</p></body></html>")
    
    png_path = Path(d) / "dst_path" / "mkdocs" / "docs_overrides" / "images"
    png_path.mkdir(parents=True)
    
    dst_path = Path(d) / "dst_path"
    await _capture_and_save_screenshot(d, dst_path)
    
    assert (png_path / "social_image.png").exists()

In [ ]:
# | export

async def _create_social_image(root_path: str, image_url: str):
    """Create social image for the project
    
    Args:
        root_path: The root path of the project.
        image_url: The image URL to be included in the social image.
    """
    html_str = _generate_html_str(root_path, image_url)
    
    with TemporaryDirectory() as d:
        
        html_path = Path(d) / "social_image.html"
        with open(html_path, "w") as f:
            f.write(html_str)
        
        if image_url == "default_social_logo.png":
            shutil.copyfile(
                Path(root_path) / "mkdocs" / "docs_overrides" / "images" / "default_social_logo.png", 
                Path(d) / "default_social_logo.png"
            )
            
        await _capture_and_save_screenshot(d, root_path)


In [ ]:
with TemporaryDirectory() as d:
    
    image_url = "default_social_logo.png"
    
    mkdocs_dir_path = Path(d) / "mkdocs" / "docs_overrides" / "images"
    mkdocs_dir_path.mkdir(parents=True)
    
    shutil.copyfile(
        Path("..") / "mkdocs" / "docs_overrides" / "images"/ "default_social_logo.png", 
        mkdocs_dir_path / "default_social_logo.png"
    )
    

    await _create_social_image(d, image_url)
    
    png_path = mkdocs_dir_path / "social_image.png"
    assert png_path.exists()

In [ ]:
# | export

def _update_social_image_in_mkdocs_yml(root_path: str, image_url: Optional[str] = None):
    """Update social image link in mkdocs yml file
    
    Args:
        root_path: The root path of the project.
        image_url: The image URL to update in the mkdocs yml file. 
    """
    
    if not image_url:
        social_image_path = Path(root_path) / "mkdocs" / "docs_overrides" / "images" / "social_image.png"
        
        if not social_image_path.exists():
            typer.secho(
                f"Unexpected error: path {social_image_path.resolve()} does not exists!",
                err=True,
                fg=typer.colors.RED,
            )
            raise typer.Exit(code=1)
        
        image_url = "overrides/images/social_image.png"
    
    yaml=YAML()
    mkdocs_yml_path = Path(root_path) / "mkdocs" / "mkdocs.yml"
    config = yaml.load(mkdocs_yml_path)
    config['extra']['social_image'] = image_url
    yaml.dump(config, mkdocs_yml_path)

In [ ]:
with TemporaryDirectory() as d:

    image_url = "default_social_logo.png"
    
    mkdocs_dir_path = Path(d) / "mkdocs" / "docs_overrides" / "images"
    mkdocs_dir_path.mkdir(parents=True)
    
    shutil.copyfile(
        Path("..") / "mkdocs" / "docs_overrides" / "images"/ "default_social_logo.png", 
        mkdocs_dir_path / "default_social_logo.png"
    )
    

    await _create_social_image(d, image_url)    
    
    mkdocs_yml_path = Path(d) / "mkdocs" 
    shutil.copyfile(Path("..") / "mkdocs" / "mkdocs.yml" , (mkdocs_yml_path / "mkdocs.yml"))
    
    image_url = "https://my-random-domain/sample.png"
    _update_social_image_in_mkdocs_yml(d, image_url)
    
    yaml=YAML()
    config = yaml.load((Path(d) / "mkdocs/mkdocs.yml"))
    print(config['extra']['social_image'])
    assert config['extra']['social_image'] == image_url, config['extra']['social_image']
    
    
    _update_social_image_in_mkdocs_yml(d)
    
    yaml=YAML()
    config = yaml.load((Path(d) / "mkdocs/mkdocs.yml"))
    print(config['extra']['social_image'])
    assert config['extra']['social_image'] == "overrides/images/social_image.png", config['extra']['social_image']
    

https://my-random-domain/sample.png
overrides/images/social_image.png


In [ ]:
# | export

def _update_social_image_in_site_overrides(root_path: str):
    """Update social image link in site_overrides HTML template
    
    Args:
        root_path: The root path of the project.
    """
    with set_cwd(root_path):
        site_overrides_path = Path(root_path) / "mkdocs" / "site_overrides" / "main.html"
        if not site_overrides_path.exists():
            typer.secho(
                f"Unexpected error: path {site_overrides_path.resolve()} does not exists!",
                err=True,
                fg=typer.colors.RED,
            )
            raise typer.Exit(code=1)
            
        with open(site_overrides_path, "r") as f:
            _new_text = f.read()
            _pattern = re.compile(r".*?{%.*?image_url = (.*)%}")
            _match = re.search(_pattern, _new_text)
            _new_text = _new_text.replace(
                _match.group(1), 'page.canonical_url ~ "" ~ config.extra.social_image ' # type: ignore
            )
        
        with open(site_overrides_path, "w") as f:
            f.write(_new_text)

In [ ]:
with TemporaryDirectory() as d:
    
    site_overrides_path = Path(d) / "mkdocs" / "site_overrides"
    site_overrides_path.mkdir(exist_ok=True, parents=True)
    shutil.copyfile(Path("..") / "mkdocs" / "site_overrides" / "main.html" , (site_overrides_path / "main.html" ))
    
    _update_social_image_in_site_overrides(d)
    
    with open((site_overrides_path / "main.html" )) as f:
        actual = f.read()
    
    print(actual)
    
    assert '{% set image_url = page.canonical_url ~ "" ~ config.extra.social_image %}' in actual, actual

{% extends "base.html" %}

{% block extrahead %}
  {% set title = config.site_name %}
  {% if page and page.meta and page.meta.title %}
    {% set title = title ~ " - " ~ page.meta.title %}
  {% elif page and page.title and not page.is_homepage %}
    {% set title = title ~ " - " ~ page.title | striptags %}
  {% endif %}
  {% set image_url = page.canonical_url ~ "" ~ config.extra.social_image %}
  <meta property="og:type" content="website" />
  <meta property="og:title" content="{{ title }}" />
  <meta property="og:description" content="{{ config.site_description }}" />
  <meta property="og:url" content="{{ page.canonical_url }}" />
  <meta property="og:image" content="{{ image_url }}" />
  <meta property="og:image:type" content="image/png" />
  <meta property="og:image:width" content="1200" />
  <meta property="og:image:height" content="600" />

  <meta name="twitter:card" content="summary_large_image" />
  <meta name="twitter:title" content="{{ title }}" />
  <meta name="twitter:desc

In [ ]:
# | export

async def generate_custom_social_image(root_path: str, prompt: str):
    """Generate a custom image for social card using the OpenAI Image API.
    
    Args:
        root_path: The root path of the project.
        prompt: The prompt to use for generating the image.
    """
    
    
    image_url = _generate_ai_image(prompt=prompt)
    
    await _create_social_image(root_path, image_url)

    _update_social_image_in_mkdocs_yml(root_path)

    _update_social_image_in_site_overrides(root_path)


In [ ]:
with TemporaryDirectory() as d:
    
    mkdocs_dir_path = Path(d) / "mkdocs" / "docs_overrides" / "images"
    mkdocs_dir_path.mkdir(exist_ok=True, parents=True)
    
    mkdocs_yml_path = Path(d) / "mkdocs" 
    shutil.copyfile(Path("..") / "mkdocs" / "mkdocs.yml" , (mkdocs_yml_path / "mkdocs.yml"))
    
    site_overrides_path = Path(d) / "mkdocs" / "site_overrides"
    site_overrides_path.mkdir(exist_ok=True, parents=True)
    shutil.copyfile(Path("..") / "mkdocs" / "site_overrides" / "main.html" , (site_overrides_path / "main.html" ))
    
    shutil.copyfile(
        Path("..") / "mkdocs" / "docs_overrides" / "images"/ "default_social_logo.png", 
        mkdocs_dir_path / "default_social_logo.png"
    )

    
    prompt = "The quick brown fox jumps over a lazy dog"
    await generate_custom_social_image(d, prompt)
    
    
    png_path = mkdocs_dir_path / "social_image.png"
    
    assert png_path.exists()
    
    
    yaml=YAML()
    config = yaml.load((Path(d) / "mkdocs/mkdocs.yml"))
    print(config['extra']['social_image'])
    assert config['extra']['social_image'] == "overrides/images/social_image.png", config['extra']['social_image']
    
    with open((site_overrides_path / "main.html" )) as f:
        actual = f.read()
    
    print(actual)
    
    assert '{% set image_url = page.canonical_url ~ "" ~ config.extra.social_image %}' in actual, actual
    

Request to OpenAI failed: Billing hard limit has been reached
Using the default social image.
overrides/images/social_image.png
{% extends "base.html" %}

{% block extrahead %}
  {% set title = config.site_name %}
  {% if page and page.meta and page.meta.title %}
    {% set title = title ~ " - " ~ page.meta.title %}
  {% elif page and page.title and not page.is_homepage %}
    {% set title = title ~ " - " ~ page.title | striptags %}
  {% endif %}
  {% set image_url = page.canonical_url ~ "" ~ config.extra.social_image %}
  <meta property="og:type" content="website" />
  <meta property="og:title" content="{{ title }}" />
  <meta property="og:description" content="{{ config.site_description }}" />
  <meta property="og:url" content="{{ page.canonical_url }}" />
  <meta property="og:image" content="{{ image_url }}" />
  <meta property="og:image:type" content="image/png" />
  <meta property="og:image:width" content="1200" />
  <meta property="og:image:height" content="600" />

  <meta name=